# Federated learning: Regression

In this notebook we are going to show how you can use a federated learning environment to create a regression model. In the notebook "Federated learning: Simple experiment" we explained the basic concepts of the framework, so now we will go slightly faster.

Firts of all we load a regression dataset that the framework already includes to allow regression experiments.

In [1]:
import shfl
from shfl.data_base.california_housing import CaliforniaHousing

database = CaliforniaHousing()
train_data, train_labels, val_data, val_labels, test_data, test_labels = database.load_data()

Using TensorFlow backend.


We are going to explore the data

In [2]:
print("Shape of train_data: " + str(train_data.shape))
print("Shape of train_labels: " + str(train_labels.shape))
print("One sample features: " + str(train_data[0]))
print("One sample label: " + str(train_labels[0]))

Shape of train_data: (16512, 8)
Shape of train_labels: (16512,)
One sample features: [ 2.00000000e+00  3.00000000e+01  2.77388256e+00  1.10692375e+00
  3.67800000e+03  3.22348817e+00  3.40900000e+01 -1.18310000e+02]
One sample label: 2.406


Federeted data generation:

In [3]:
import shfl

iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_label = iid_distribution.get_federated_data("hotels_california", 20, percent=10)

Model definition:

In [4]:
import keras

def model_builder():
    # create model
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=["mae"])
    
    return shfl.model.DeepLearningModel(model)

Federated environment definition:

In [5]:
aggregator = shfl.federated_aggregator.AvgFedAggregator()
federated_government = shfl.learning_approach.FederatedGovernment(model_builder, federated_data, aggregator)

Reshaping data:

In [6]:
import numpy as np

class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        labeled_data.label = np.reshape(labeled_data.label, (labeled_data.label.shape[0], 1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

Running experiment:

In [7]:
test_label = np.reshape(test_label, (test_label.shape[0], 1))
federated_government.run_rounds(3, test_data, test_label)

Accuracy round 0
Test performance client <shfl.private.node.DataNode object at 0x1485e5d90>: [3.7208566212838936, 1.5233341455459595]
Test performance client <shfl.private.node.DataNode object at 0x10f9e4150>: [4.207773434099301, 1.6796486377716064]
Test performance client <shfl.private.node.DataNode object at 0x149166f50>: [4.00747069092684, 1.6168088912963867]
Test performance client <shfl.private.node.DataNode object at 0x1297fda50>: [3.7033289190410645, 1.5174965858459473]
Test performance client <shfl.private.node.DataNode object at 0x1297fd8d0>: [4.147247159203817, 1.6607636213302612]
Test performance client <shfl.private.node.DataNode object at 0x1297fd1d0>: [4.038172195124072, 1.6265453100204468]
Test performance client <shfl.private.node.DataNode object at 0x1297fd250>: [3.800459671390149, 1.5497686862945557]
Test performance client <shfl.private.node.DataNode object at 0x1297fd3d0>: [3.6900354448214983, 1.5130497217178345]
Test performance client <shfl.private.node.DataNode o